# Quick Tutorial for the Revised Tensor Network Code
Here's a quick overview of how to use the revised code for optimizing a tensor network (TN) with fixed ranks _(credit: Michelle for putting together the initial version of the code)_. The current version is a bit more modular, so that we can use the same continuous optimization routine for each of the problems being solved. The ingredients that still need to be implemented are:

1. Code implementing the different discrete optimization procedures
2. Loss function for the tensor completion task, which takes in our TN and dataset of known tensor elements, and returns a loss based on the average loss in each of these known elements

I'll talk about those some more at the end of this, but let's first see how we interface with the continuous optimization code!

In [1]:
import sys

import torch
import tensornetwork as tn

# Make sure notebook can find the core code
sys.path.append("..")
import core_code as cc

# Set random seed for reproducibility
torch.manual_seed(0)

### Initializing TNs
Tensor networks of arbitrary rank are initialized with the `random_tn` function, with the rank being specified in one of several ways. Scalar values of the `rank` argument give constant-rank TNs (default: rank=1), but individual ranks are also possible using a list as input. Note how the ranks are being displayed, with upper diagonal entries giving the TN ranks, and diagonal entries giving the input dimension of each core _(credit: Meraj for the idea of putting the input dims on the diagonals)_

In [2]:
# The dimensions of each of the inputs to our tensor network (TN)
input_dims = [2, 4, 5, 6]

# Initialize a random rank-1 TN
example_tn = cc.random_tn(input_dims)

print("Rank-1 TN has ranks")
cc.print_ranks(example_tn)
print("...and input dimensions")
print(cc.get_indims(example_tn))

Rank-1 TN has ranks
tensor([[2, 1, 1, 1],
        [0, 4, 1, 1],
        [0, 0, 5, 1],
        [0, 0, 0, 6]])
...and input dimensions
(2, 4, 5, 6)


In [3]:
# Random TNs with higher ranks can also be defined
base_tn = cc.random_tn(input_dims, rank=3)

print("Rank-3 TN has ranks")
cc.print_ranks(base_tn)
print()

# Individual ranks of TN edges can be set with upper-triangular format
rank_list = [[1,1,2], 
               [3,5], 
                 [8]]
weird_tn = cc.random_tn(input_dims, rank=rank_list)

print("Weird TN has ranks")
cc.print_ranks(weird_tn)
print(f"This network is defined by {cc.num_params(weird_tn)} real-valued parameters")

Rank-3 TN has ranks
tensor([[2, 3, 3, 3],
        [0, 4, 3, 3],
        [0, 0, 5, 3],
        [0, 0, 0, 6]])

Weird TN has ranks
tensor([[2, 1, 1, 2],
        [0, 4, 3, 5],
        [0, 0, 5, 8],
        [0, 0, 0, 6]])
This network is defined by 664 real-valued parameters


### Tensor Recovery

Let's now see how we solve the continuous optimization for a tensor recovery problem. The key difference with the earlier version of the code is that a loss function is input to `continuous_optim`, which specifies the type of problem being solved.

In this case, we use `tensor_recovery_loss`, but generally the loss function must take in the TN being trained and a problem-dependent data format, and return a loss. In other words:

`loss_value = loss_fun(our_tn, target_data)`

In [4]:
# To train, the tensor network cores must first be made trainable
base_tn = cc.make_trainable(base_tn)

# Initialize a random target tensor of rank 4, which we will minimize
# L2 distance with (base_tn is only rank-3, so loss won't reach 0)
goal_tn = cc.random_tn(input_dims, rank=4)

# continuous_optim requires the following as input:
# (1) A tensor network model, base_tn
# (2) A target dataset, in this case just goal_tn
# (3) A loss function of the form loss_fun(base_tn, batch), where 
#     batch is a minibatch of training data (just goal_tn here)
# The choice of (2)+(3) fully determines the learning task, with other 
# problems taking regular datasets as inputs

# For tensor recovery, use tensor_recovery_loss from core_code module
loss_fun = cc.tensor_recovery_loss
trained_tn, init_loss, final_loss = cc.continuous_optim(base_tn, goal_tn, 
                                                        loss_fun)
print(f"Train loss went from {init_loss:.3f} to {final_loss:.3f} in 10 epochs\n")

# Note that trained_tn gives the model after training, which will be 
# needed for discrete optimization algorithm. To continue training the 
# trained model, just run the same code above, but with trained_tn as input
_, init_loss, final_loss = cc.continuous_optim(trained_tn, goal_tn, loss_fun)
print("Note how the loss continued decreasing from where it had left off")

  EPOCH 1 
    Train loss: 14382.888
  EPOCH 2 
    Train loss: 14268.451
  EPOCH 3 
    Train loss: 14162.802
  EPOCH 4 
    Train loss: 14064.630
  EPOCH 5 
    Train loss: 13972.844
  EPOCH 6 
    Train loss: 13886.522
  EPOCH 7 
    Train loss: 13804.878
  EPOCH 8 
    Train loss: 13727.241
  EPOCH 9 
    Train loss: 13653.029
  EPOCH 10 
    Train loss: 13581.736

Train loss went from 14382.888 to 13581.736 in 10 epochs

  EPOCH 1 
    Train loss: 13512.916
  EPOCH 2 
    Train loss: 13446.178
  EPOCH 3 
    Train loss: 13381.173
  EPOCH 4 
    Train loss: 13317.587
  EPOCH 5 
    Train loss: 13255.141
  EPOCH 6 
    Train loss: 13193.579
  EPOCH 7 
    Train loss: 13132.670
  EPOCH 8 
    Train loss: 13072.202
  EPOCH 9 
    Train loss: 13011.981
  EPOCH 10 
    Train loss: 12951.827

Note how the loss continued decreasing from where it had left off


### Customizing the Continuous Optimization Procedure

Although we can't directly tweak the continuous_optim code for different problem types, we still have a lot of flexibility owing to the `other_args` argument (a dictionary). Let's explore the current options for `other_args`, and more options can be added later if needed.

In [5]:
# 10 epochs is the default, but you can change this
cc.continuous_optim(base_tn, goal_tn, loss_fun, epochs=2)

# Feeding a dictionary as other_args arg of continuous_optim lets you 
# control the learning rate and optimizer (chosen from torch.optim)
print("Does Adam do any better than SGD? Let's increase the learning "
      "rate for good measure!")
fast_adam = {'optim': 'Adam',   # Default: 'SGD'
              'lr':   1e-2}     # Default: 1e-3
_ = cc.continuous_optim(base_tn, goal_tn, loss_fun, other_args=fast_adam)

# You can also run optimization silently via the `print` argument
silent = {'print': False}      # Default: True
print("Beginning silent training...")
_, init_loss, final_loss = cc.continuous_optim(base_tn, goal_tn, loss_fun, 
                                               other_args=silent)
print("Silent training finished")
print(f"Train loss went from {init_loss:.3f} to {final_loss:.3f} in 10 epochs\n")

# For tensor recovery, there is only one item in our dataset (goal_tn), 
# leading to only one gradient step per epoch. Using the `reps` argument 
# can reduce printing by going through training data many times per epoch
print("For recovery, it's useful to go through the dataset many times per epoch")
print("Note: This is just a trick to avoid excessive printing")
lotsa_reps = {'reps': 100}
_ = cc.continuous_optim(base_tn, goal_tn, loss_fun, other_args=lotsa_reps)

  EPOCH 1 
    Train loss: 14382.888
  EPOCH 2 
    Train loss: 14268.451

Does Adam do any better than SGD? Let's increase the learning rate for good measure!
  EPOCH 1 
    Train loss: 14382.888
  EPOCH 2 
    Train loss: 14329.778
  EPOCH 3 
    Train loss: 14277.630
  EPOCH 4 
    Train loss: 14226.434
  EPOCH 5 
    Train loss: 14176.182
  EPOCH 6 
    Train loss: 14126.857
  EPOCH 7 
    Train loss: 14078.434
  EPOCH 8 
    Train loss: 14030.883
  EPOCH 9 
    Train loss: 13984.173
  EPOCH 10 
    Train loss: 13938.271

Beginning silent training...
Silent training finished
Train loss went from 14382.888 to 13581.736 in 10 epochs

For recovery, it's useful to go through the dataset many times per epoch
Note: This is just a trick to avoid excessive printing
  EPOCH 1 (100 reps)
    Train loss: 10491.699
  EPOCH 2 (100 reps)
    Train loss: 2719.662
  EPOCH 3 (100 reps)
    Train loss: 1172.866
  EPOCH 4 (100 reps)
    Train loss: 1015.489
  EPOCH 5 (100 reps)
    Train loss: 917.84

### Regression of Scalar-Valued Function

Continuous optimization for function regression works exactly the same as for tensor recovery, but with a different loss function and target dataset format. To generate this data from a TN, use `generate_regression_data`

In [6]:
# The generate_regression_data function takes in a target TN and a dataset
# size, and produces a pair of random inputs and associated (noisy) outputs
num_train = 10000
train_data = cc.generate_regression_data(goal_tn, num_train)

# For regression, use cc.regression_loss
loss_fun = cc.regression_loss
_ = cc.continuous_optim(base_tn, train_data, loss_fun)

# Since we're doing machine learning, it's good to have a held-out validation 
# set to determine loss, early stopping, etc. This is easy to do
num_val = 1000
val_data = cc.generate_regression_data(goal_tn, num_train)
print("Same training process, but with validation data")
_ = cc.continuous_optim(base_tn, train_data, loss_fun, val_data=val_data)

# It appears there's a lot of overfitting going on! In this case, we can
# use the validation loss to choose the stopping time, which is done by
# setting epochs=None in continuous_optim
_ = cc.continuous_optim(base_tn, train_data, loss_fun, val_data=val_data, 
                        epochs=None)

  EPOCH 1 
    Train loss: 9288.677
  EPOCH 2 
    Train loss: 7940.358
  EPOCH 3 
    Train loss: 7048.312
  EPOCH 4 
    Train loss: 6696.229
  EPOCH 5 
    Train loss: 6518.609
  EPOCH 6 
    Train loss: 6365.256
  EPOCH 7 
    Train loss: 6225.493
  EPOCH 8 
    Train loss: 6094.021
  EPOCH 9 
    Train loss: 5967.562
  EPOCH 10 
    Train loss: 5844.089

Same training process, but with validation data
  EPOCH 1 
    Val. loss:  12876.681
    Train loss: 9288.677
  EPOCH 2 
    Val. loss:  12841.797
    Train loss: 7940.358
  EPOCH 3 
    Val. loss:  12830.151
    Train loss: 7048.312
  EPOCH 4 
    Val. loss:  12770.275
    Train loss: 6696.229
  EPOCH 5 
    Val. loss:  12637.485
    Train loss: 6518.609
  EPOCH 6 
    Val. loss:  12509.603
    Train loss: 6365.256
  EPOCH 7 
    Val. loss:  12402.265
    Train loss: 6225.493
  EPOCH 8 
    Val. loss:  12309.287
    Train loss: 6094.021
  EPOCH 9 
    Val. loss:  12226.905
    Train loss: 5967.562
  EPOCH 10 
    Val. loss:  1215

    Train loss: 2577.158
  EPOCH 118 
    Val. loss:  9568.142
    Train loss: 1729.716
  EPOCH 119 
    Val. loss:  9589.182
    Train loss: 2596.337
  EPOCH 120 
    Val. loss:  9564.695
    Train loss: 1675.944
  EPOCH 121 
    Val. loss:  9599.379
    Train loss: 2549.184
  EPOCH 122 
    Val. loss:  9563.654
    Train loss: 1696.591
  EPOCH 123 
    Val. loss:  9597.237
    Train loss: 2571.765
  EPOCH 124 
    Val. loss:  9561.144
    Train loss: 1640.881
  EPOCH 125 
    Val. loss:  9608.439
    Train loss: 2522.960
  EPOCH 126 
    Val. loss:  9560.757
    Train loss: 1665.519
  EPOCH 127 
    Val. loss:  9606.069
    Train loss: 2549.688
  EPOCH 128 
    Val. loss:  9559.160
    Train loss: 1606.761
  EPOCH 129 
    Val. loss:  9618.459
    Train loss: 2498.087
  EPOCH 130 
    Val. loss:  9559.399
    Train loss: 1636.479
  EPOCH 131 
    Val. loss:  9615.641
    Train loss: 2529.903
  EPOCH 132 
    Val. loss:  9558.700
    Train loss: 1573.366
  EPOCH 133 
    Val. loss:  9

    Train loss: 1764.335
  EPOCH 250 
    Val. loss:  9936.434
    Train loss: 1625.947
  EPOCH 251 
    Val. loss:  10012.206
    Train loss: 2397.942
  EPOCH 252 
    Val. loss:  9987.514
    Train loss: 809.251
  EPOCH 253 
    Val. loss:  10159.693
    Train loss: 1762.568
  EPOCH 254 
    Val. loss:  9946.689
    Train loss: 1611.364
  EPOCH 255 
    Val. loss:  10019.355
    Train loss: 2390.993
  EPOCH 256 
    Val. loss:  9996.871
    Train loss: 799.391
  EPOCH 257 
    Val. loss:  10165.309
    Train loss: 1763.351
  EPOCH 258 
    Val. loss:  9957.180
    Train loss: 1594.291
  EPOCH 259 
    Val. loss:  10026.614
    Train loss: 2383.314
  EPOCH 260 
    Val. loss:  10006.199
    Train loss: 791.462
  EPOCH 261 
    Val. loss:  10169.895
    Train loss: 1770.798
  EPOCH 262 
    Val. loss:  9968.554
    Train loss: 1569.935
  EPOCH 263 
    Val. loss:  10034.841
    Train loss: 2373.656
  EPOCH 264 
    Val. loss:  10015.719
    Train loss: 788.068
  EPOCH 265 
    Val. los

    Train loss: 611.077
  EPOCH 381 
    Val. loss:  10164.733
    Train loss: 1889.774
  EPOCH 382 
    Val. loss:  9946.103
    Train loss: 1087.293
  EPOCH 383 
    Val. loss:  10049.013
    Train loss: 2267.441
  EPOCH 384 
    Val. loss:  9981.395
    Train loss: 583.018
  EPOCH 385 
    Val. loss:  10132.720
    Train loss: 1831.310
  EPOCH 386 
    Val. loss:  9963.383
    Train loss: 1178.480
  EPOCH 387 
    Val. loss:  9992.874
    Train loss: 2268.299
  EPOCH 388 
    Val. loss:  10001.391
    Train loss: 596.652
  EPOCH 389 
    Val. loss:  10051.815
    Train loss: 1915.025
  EPOCH 390 
    Val. loss:  10011.875
    Train loss: 1128.392
  EPOCH 391 
    Val. loss:  9935.209
    Train loss: 2219.299
  EPOCH 392 
    Val. loss:  10031.640
    Train loss: 716.128
  EPOCH 393 
    Val. loss:  9932.694
    Train loss: 2085.561
  EPOCH 394 
    Val. loss:  10057.692
    Train loss: 898.779
  EPOCH 395 
    Val. loss:  9872.649
    Train loss: 2122.293
  EPOCH 396 
    Val. loss:

    Train loss: 475.326
  EPOCH 513 
    Val. loss:  9582.212
    Train loss: 2022.927
  EPOCH 514 
    Val. loss:  9956.601
    Train loss: 589.318
  EPOCH 515 
    Val. loss:  9591.546
    Train loss: 2244.528
  EPOCH 516 
    Val. loss:  9899.237
    Train loss: 355.558
  EPOCH 517 
    Val. loss:  9782.295
    Train loss: 1354.104
  EPOCH 518 
    Val. loss:  9973.138
    Train loss: 1626.673
  EPOCH 519 
    Val. loss:  9678.788
    Train loss: 2009.984
  EPOCH 520 
    Val. loss:  9910.551
    Train loss: 632.218
  EPOCH 521 
    Val. loss:  9673.310
    Train loss: 2255.706
  EPOCH 522 
    Val. loss:  9849.294
    Train loss: 357.588
  EPOCH 523 
    Val. loss:  9880.800
    Train loss: 1388.610
  EPOCH 524 
    Val. loss:  9834.015
    Train loss: 1548.742
  EPOCH 525 
    Val. loss:  9712.837
    Train loss: 1988.895
  EPOCH 526 
    Val. loss:  9869.242
    Train loss: 645.586
  EPOCH 527 
    Val. loss:  9681.872
    Train loss: 2255.433
  EPOCH 528 
    Val. loss:  9821.27

KeyboardInterrupt: 

In [ ]:
# Mini-experiment to determine usefulness of different optimizers

candidate_optims = ['Adadelta', 'Adagrad', 'Adam', 'AdamW', 
                    'Adamax', 'RMSprop', 'Rprop', 'SGD']
my_args = {'lr': 1e-3}
loss_fun = cc.tensor_recovery_loss
percent_dec = {}
my_args = {'print': False}
print("Testing optimizers...")
for optim in candidate_optims:
    print("  " + optim)
    my_args['optim'] = optim
    _, loss_i, loss_f = cc.continuous_optim(base_tn, goal_tn, loss_fun, 
                                            epochs=100, other_args=my_args)
    percent_dec[optim] = 100 * (loss_i-loss_f) / (loss_i)

print("The Rankings in Loss Decrease are")
percent_dec = sorted(percent_dec.items(), key=lambda p_dec: p_dec[1])
for optim, p_dec in percent_dec:
    print(f"  {optim+':':<9} {p_dec:.2f}% decrease")
          
print("\nLooks like Rprop does really well, although I'm not sure why!")